In [1]:
# plot the audio data
# use popup window
# %matplotlib qt
import matplotlib.pyplot as plt

from utils import *

chrip freq 5000, 10000.0
chrip freq 7000, 11000.0


In [2]:
rate = 48000
freq = 1000
samples = rate // freq
channels = 12

def encode_one(bits):
    assert len(bits) == channels
    freq_amp = np.zeros(samples//2+1, dtype=np.complex128)
    A = samples * 0.5 / channels
    for i in range(channels):
        if bits[i]:
            freq_amp[i + 1] = -A * 1j
        else:
            freq_amp[i + 1] = A

    wave = np.fft.irfft(freq_amp)
    return wave

def decode_one(wave):
    freq_amp = np.fft.rfft(wave)
    bits = []
    for i in range(channels):
        if (np.abs(np.imag(freq_amp[i + 1])) > np.abs(np.real(freq_amp[i + 1]))):
            bits.append(1)
        else:
            bits.append(0)
    return np.array(bits)

def encode_many(bits):
    assert len(bits) % channels == 0
    cp_samples = 12
    waves = []
    for i in range(0, len(bits), channels):
        wave = encode_one(bits[i:i+channels])
        wave = np.concatenate([wave[-cp_samples:], wave])
        assert len(wave) == samples + cp_samples
        waves.append(wave)
    wave = np.concatenate(waves)
    assert len(wave) == len(bits) // channels * (samples + cp_samples)
    return wave

def decode_many(waves):
    cp_samples = 12
    bits = []
    for i in range(0, len(waves), cp_samples + samples):
        wave = waves[i:i+cp_samples+samples]
        assert len(wave) == samples + cp_samples
        bits.append(decode_one(wave[cp_samples:]))
    return np.concatenate(bits)

similarities = []
for i in range(10):
    data_len = 100
    bits = np.random.randint(0, 2, channels * data_len)
    wave = encode_many(bits)
    # print(bits)
    # print(decode_many(wave))

    # print(len(wave))
    record_wave, rate, raw_wave = play_and_record_precise2(wave, rate)
    # print(len(record_wave))

    decoded_bits = decode_many(record_wave)

    similarity = np.sum(decoded_bits == bits) / len(bits)

    # bps = len(bits) / (len(wave) / rate)
    # 10105.263157894737

    print(similarity)
    similarities.append(similarity)

one_rate = similarities.count(1) / len(similarities)
print(one_rate)

# print(np.mean(similarities))

0.5166666666666667
0.7975
0.8241666666666667
1.0
1.0
1.0
1.0
0.9991666666666666
0.7433333333333333
1.0
0.5


In [ ]:
similarities = []
for i in range(1):
    signal = np.loadtxt(f"/home/ryo/code/supersonic/build/ddata/frames{i}.txt")
    recv_signal = np.loadtxt(f"/home/ryo/code/supersonic/build/ddata/recv_frames{i}.txt")

    bits = decode_many(signal)
    recv_bits = decode_many(recv_signal)

    similarity = np.sum(recv_bits == bits) / len(bits)
    # print(similarity)
    similarities.append(similarity)
one_rate = similarities.count(1) / len(similarities)
print(one_rate)

0.84
